In [48]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.ensemble import GradientBoostingRegressor
import category_encoders as ce
from sklearn.tree import plot_tree
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import plotly.express as px
import plotly.graph_objects as go
from sklearn.pipeline import make_pipeline


In [2]:
df = pd.read_csv(r'C:\Users\samina\Desktop\GA_DATA_SCIENCE\Data_Sci\Homework\Unit3\data\insurance_premiums.csv')
#df.head(20)

In [ ]:
te = ce.TargetEncoder(cols=['smoker'])
ohe = ce.OneHotEncoder(cols=['region', 'sex'], verbose=1)

#Debugging encoder
le = preprocessing.LabelEncoder()        #label encoder
# Transform the sex column
df['sex'] = le.fit_transform(df['sex'])
df['smoker'] = le.fit_transform(df['smoker'])
df['region'] = le.fit_transform(df['region'])
gbm = GradientBoostingRegressor(verbose=1)  

pipe = make_pipeline(le, gbm)
#pipe = make_pipeline(te, ohe, gbm)            #pipe 2 uses Gradient boosting
print(ohe)
print(te)
print(pipe)

print(le)
df.head(10)

In [4]:
'''
test = df.groupby('age').apply(lambda x: x.iloc[-30:])
train = df.groupby('age').apply(lambda x: x.iloc[:-30])  #iloc --> everything from start except last 30
'''

"\ntest = df.groupby('age').apply(lambda x: x.iloc[-30:])\ntrain = df.groupby('age').apply(lambda x: x.iloc[:-30])  #iloc --> everything from start except last 30\n"

In [5]:
#train.reset_index(drop=True, inplace=True)

In [6]:
'''
val = train.groupby('age').apply(lambda x: x.iloc[-30:])     #validation set
#val.reset_index(drop=True, inplace=True)
train = train.groupby('age').apply(lambda x : x.iloc[:-30])  #stop from leaking data
'''

"\nval = train.groupby('age').apply(lambda x: x.iloc[-30:])     #validation set\n#val.reset_index(drop=True, inplace=True)\ntrain = train.groupby('age').apply(lambda x : x.iloc[:-30])  #stop from leaking data\n"

In [97]:
'''
# FUNCTION USED FROM LAB FROM CLASS
#USE FOR RESTAURANTS DATASET OR SIMILAR DATASETS
# define some functions that we can reuse

#parameter: df -->insurance data
#           val_units --> value used to exclude from train n valdiation data
#           return_val --> bool value if false func return train and test data else returns train, test n validation data

def create_val_splits(df, val_units=30, return_val=False):
    """Function that will take in a dataset and split it up into training, validation, and test sets"""
    # split into training, validation, and test sets
    #df = df.drop('charges', axis=1)
    train = df.groupby('age').apply(lambda x: x.iloc[:-val_units]).reset_index(drop=True)
    test  = df.groupby('age').apply(lambda x: x.iloc[-val_units:]).reset_index(drop=True)
    
    if return_val:
        val   = train.groupby('age').apply(lambda x: x.iloc[-val_units:]).reset_index(drop=True)
        train = train.groupby('age').apply(lambda x: x.iloc[:-val_units]).reset_index(drop=True)
        return train, val, test
    else:
        return train, test
'''

def split_dataset(df, split_frac=0.2, return_val = True, return_test = False, random_state = 7):
    X = df.drop('charges', axis=1)
    y = df['charges']
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = split_frac, random_state = random_state)
    #pipe_line[0].fit_transform(X_train, y_train)
    #pipe_line[1].fit_transform(X_train, y_train)
    
    #pipe_line[0].fit_transform(X_test, y_test)
    #pipe_line[1].fit_transform(X_test, y_test)
    
    
    if return_val:
        X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = split_frac, random_state = random_state)
        #pipe_line[0].fit_transform(X_val, y_val)
        #pipe_line[1].fit_transform(X_val, y_val)

    if return_val and return_test:
        return X_train, y_train, X_val, y_val, X_test, y_test
    
    if return_val:
        return X_train, y_train, X_val, y_val


In [98]:
#train, val, test = create_val_splits(df, return_val=True)
X_train, y_train, X_val, y_val, X_test, y_test = split_dataset(df, split_frac=0.15, return_val=True, return_test=True)
#pipe[0].fit_transform(X_val, y_val)

In [100]:
X_val
#X_train , y_train = train.drop('charges', axis=1), train['charges']         
#X_val, y_val = val.drop('charges', axis=1), val['charges']

,age,sex,bmi,children,smoker,region
1268,20,0,33.30,0,0,3
735,49,0,34.77,1,0,1
214,45,0,30.90,2,0,3
145,29,0,38.83,3,0,2
841,59,1,24.70,0,0,0
...,...,...,...,...,...,...
1076,47,0,32.00,1,0,3
953,44,1,30.20,2,1,3
951,51,1,42.90,2,1,2
803,18,0,42.24,0,1,2


In [102]:
pipe.fit(X_train, y_train)

TypeError: fit_transform() takes 2 positional arguments but 3 were given

In [103]:
pipe.predict(X_val)

ValueError: y should be a 1d array, got an array of shape (171, 6) instead.

In [85]:
pipe.score(X_val, y_val)

0.8651976600091671

In [86]:
pipe[-1].set_params(criterion='mae', max_depth=3)

GradientBoostingRegressor(criterion='mae', verbose=1)

In [132]:
def scores(criteria, loss_function, tree_depth, boosting_rounds):
    scores_for_criteria = []
    i = 0.01             # i --> learning rate
    while i <= 0.1:
        gbm = GradientBoostingRegressor(criterion=criteria, loss=loss_function, max_depth=tree_depth, learning_rate=i, n_estimators=boosting_rounds)
        gbm.fit(X_train, y_train)
        scores_for_criteria.append( (criteria, loss_function, tree_depth, boosting_rounds, i, gbm.score(X_val, y_val))) 
        i += 0.01
    return scores_for_criteria

In [ ]:
#Manipulate these paramenters
# loss=' {'ls', 'lad', 'huber', 'quantile'},
#learning_rate=0.01, 0.05, 0.1, 
#n_estimators=100, <-- Change default
#criterion='{'friedman_mse', 'mse', 'mae'}',
#max_depth --> 10.704 --> 11
#loss{‘deviance’, ‘exponential’}, default=’deviance’

loss_func = ['ls', 'lad', 'huber', 'quantile']
criteria = ['friedman_mse', 'mse', 'mae']
tree_depths = [i for i in range(1, 5)]                              #df.size * 0.001
boosting = [i for i in range(100, 900, 150)]

gbm_scores_mse = []
gbm_scores_friedman_mse = []
gbm_scores_mae = []

#i --> criteria
for i in criteria:
    #j --> loss func
    for j in loss_func:
        #k --> tree depth
        for k in tree_depths:
            for l in boosting:
                if i == 'mse':
                    gbm_scores_mse.append(scores(i, j, k, l))
                    print(f"Criteria:{i}\tLoss Func: {j}\tTree Depth: {k}\tBoosting round: {l}")
                elif i == 'friedman_mse':
                    gbm_scores_friedman_mse.append(scores(i, j, k, l))
                    print(f"Criteria:{i}\tLoss Func: {j}\tTree Depth: {k}\tBoosting round: {l}")
                else:
                    gbm_scores_mae.append(scores(i, j, k, l))
                    print(f"Criteria:{i}\tLoss Func: {j}\tTree Depth: {k}\tBoosting round: {l}")

Criteria:friedman_mse	Loss Func: ls	Tree Depth: 1	Boosting round: 100
Criteria:friedman_mse	Loss Func: ls	Tree Depth: 1	Boosting round: 250
Criteria:friedman_mse	Loss Func: ls	Tree Depth: 1	Boosting round: 400
Criteria:friedman_mse	Loss Func: ls	Tree Depth: 1	Boosting round: 550
Criteria:friedman_mse	Loss Func: ls	Tree Depth: 1	Boosting round: 700
Criteria:friedman_mse	Loss Func: ls	Tree Depth: 1	Boosting round: 850
Criteria:friedman_mse	Loss Func: ls	Tree Depth: 2	Boosting round: 100
Criteria:friedman_mse	Loss Func: ls	Tree Depth: 2	Boosting round: 250
Criteria:friedman_mse	Loss Func: ls	Tree Depth: 2	Boosting round: 400
Criteria:friedman_mse	Loss Func: ls	Tree Depth: 2	Boosting round: 550
Criteria:friedman_mse	Loss Func: ls	Tree Depth: 2	Boosting round: 700
Criteria:friedman_mse	Loss Func: ls	Tree Depth: 2	Boosting round: 850
Criteria:friedman_mse	Loss Func: ls	Tree Depth: 3	Boosting round: 100
Criteria:friedman_mse	Loss Func: ls	Tree Depth: 3	Boosting round: 250
Criteria:friedman_ms

Criteria:mse	Loss Func: ls	Tree Depth: 4	Boosting round: 550
Criteria:mse	Loss Func: ls	Tree Depth: 4	Boosting round: 700
Criteria:mse	Loss Func: ls	Tree Depth: 4	Boosting round: 850
Criteria:mse	Loss Func: lad	Tree Depth: 1	Boosting round: 100
Criteria:mse	Loss Func: lad	Tree Depth: 1	Boosting round: 250
Criteria:mse	Loss Func: lad	Tree Depth: 1	Boosting round: 400
Criteria:mse	Loss Func: lad	Tree Depth: 1	Boosting round: 550
Criteria:mse	Loss Func: lad	Tree Depth: 1	Boosting round: 700
Criteria:mse	Loss Func: lad	Tree Depth: 1	Boosting round: 850
Criteria:mse	Loss Func: lad	Tree Depth: 2	Boosting round: 100
Criteria:mse	Loss Func: lad	Tree Depth: 2	Boosting round: 250
Criteria:mse	Loss Func: lad	Tree Depth: 2	Boosting round: 400
Criteria:mse	Loss Func: lad	Tree Depth: 2	Boosting round: 550
Criteria:mse	Loss Func: lad	Tree Depth: 2	Boosting round: 700
Criteria:mse	Loss Func: lad	Tree Depth: 2	Boosting round: 850
Criteria:mse	Loss Func: lad	Tree Depth: 3	Boosting round: 100
Criteria:ms

F:\ANACONDA\lib\site-packages\sklearn\ensemble\_gb.py:1630: FutureWarning: criterion='mae' was deprecated in version 0.24 and will be removed in version 1.1 (renaming of 0.26). The correct way of minimizing the absolute error is to use  loss='lad' instead.
  warnings.warn("criterion='mae' was deprecated in version 0.24 and "
F:\ANACONDA\lib\site-packages\sklearn\ensemble\_gb.py:1630: FutureWarning: criterion='mae' was deprecated in version 0.24 and will be removed in version 1.1 (renaming of 0.26). The correct way of minimizing the absolute error is to use  loss='lad' instead.
  warnings.warn("criterion='mae' was deprecated in version 0.24 and "
F:\ANACONDA\lib\site-packages\sklearn\ensemble\_gb.py:1630: FutureWarning: criterion='mae' was deprecated in version 0.24 and will be removed in version 1.1 (renaming of 0.26). The correct way of minimizing the absolute error is to use  loss='lad' instead.
  warnings.warn("criterion='mae' was deprecated in version 0.24 and "
F:\ANACONDA\lib\sit

Criteria:mae	Loss Func: ls	Tree Depth: 1	Boosting round: 100


F:\ANACONDA\lib\site-packages\sklearn\ensemble\_gb.py:1630: FutureWarning: criterion='mae' was deprecated in version 0.24 and will be removed in version 1.1 (renaming of 0.26). The correct way of minimizing the absolute error is to use  loss='lad' instead.
  warnings.warn("criterion='mae' was deprecated in version 0.24 and "
F:\ANACONDA\lib\site-packages\sklearn\ensemble\_gb.py:1630: FutureWarning: criterion='mae' was deprecated in version 0.24 and will be removed in version 1.1 (renaming of 0.26). The correct way of minimizing the absolute error is to use  loss='lad' instead.
  warnings.warn("criterion='mae' was deprecated in version 0.24 and "
F:\ANACONDA\lib\site-packages\sklearn\ensemble\_gb.py:1630: FutureWarning: criterion='mae' was deprecated in version 0.24 and will be removed in version 1.1 (renaming of 0.26). The correct way of minimizing the absolute error is to use  loss='lad' instead.
  warnings.warn("criterion='mae' was deprecated in version 0.24 and "
F:\ANACONDA\lib\sit

...

F:\ANACONDA\lib\site-packages\sklearn\ensemble\_gb.py:1630: FutureWarning: criterion='mae' was deprecated in version 0.24 and will be removed in version 1.1 (renaming of 0.26). The correct way of minimizing the absolute error is to use  loss='lad' instead.
  warnings.warn("criterion='mae' was deprecated in version 0.24 and "
F:\ANACONDA\lib\site-packages\sklearn\ensemble\_gb.py:1630: FutureWarning: criterion='mae' was deprecated in version 0.24 and will be removed in version 1.1 (renaming of 0.26). The correct way of minimizing the absolute error is to use  loss='lad' instead.
  warnings.warn("criterion='mae' was deprecated in version 0.24 and "
F:\ANACONDA\lib\site-packages\sklearn\ensemble\_gb.py:1630: FutureWarning: criterion='mae' was deprecated in version 0.24 and will be removed in version 1.1 (renaming of 0.26). The correct way of minimizing the absolute error is to use  loss='lad' instead.
  warnings.warn("criterion='mae' was deprecated in version 0.24 and "
F:\ANACONDA\lib\sit

Criteria:mae	Loss Func: ls	Tree Depth: 4	Boosting round: 550


F:\ANACONDA\lib\site-packages\sklearn\ensemble\_gb.py:1630: FutureWarning: criterion='mae' was deprecated in version 0.24 and will be removed in version 1.1 (renaming of 0.26). The correct way of minimizing the absolute error is to use  loss='lad' instead.
  warnings.warn("criterion='mae' was deprecated in version 0.24 and "
F:\ANACONDA\lib\site-packages\sklearn\ensemble\_gb.py:1630: FutureWarning: criterion='mae' was deprecated in version 0.24 and will be removed in version 1.1 (renaming of 0.26). The correct way of minimizing the absolute error is to use  loss='lad' instead.
  warnings.warn("criterion='mae' was deprecated in version 0.24 and "
F:\ANACONDA\lib\site-packages\sklearn\ensemble\_gb.py:1630: FutureWarning: criterion='mae' was deprecated in version 0.24 and will be removed in version 1.1 (renaming of 0.26). The correct way of minimizing the absolute error is to use  loss='lad' instead.
  warnings.warn("criterion='mae' was deprecated in version 0.24 and "
F:\ANACONDA\lib\sit

Criteria:mae	Loss Func: ls	Tree Depth: 4	Boosting round: 700


F:\ANACONDA\lib\site-packages\sklearn\ensemble\_gb.py:1630: FutureWarning: criterion='mae' was deprecated in version 0.24 and will be removed in version 1.1 (renaming of 0.26). The correct way of minimizing the absolute error is to use  loss='lad' instead.
  warnings.warn("criterion='mae' was deprecated in version 0.24 and "
F:\ANACONDA\lib\site-packages\sklearn\ensemble\_gb.py:1630: FutureWarning: criterion='mae' was deprecated in version 0.24 and will be removed in version 1.1 (renaming of 0.26). The correct way of minimizing the absolute error is to use  loss='lad' instead.
  warnings.warn("criterion='mae' was deprecated in version 0.24 and "
F:\ANACONDA\lib\site-packages\sklearn\ensemble\_gb.py:1630: FutureWarning: criterion='mae' was deprecated in version 0.24 and will be removed in version 1.1 (renaming of 0.26). The correct way of minimizing the absolute error is to use  loss='lad' instead.
  warnings.warn("criterion='mae' was deprecated in version 0.24 and "
F:\ANACONDA\lib\sit

In [119]:
gbm_scores_mae

[]